# Gesture Recognition
In this group project, you are going to build a 3D Conv model that will be able to predict the 5 gestures correctly. Please import the following libraries to get started.

In [1]:
import numpy as np
import os
from scipy.misc import imread, imresize
import datetime
import os
import matplotlib as plt
from skimage import io
from skimage.transform import rescale, resize
import cv2

We set the random seed so that the results don't vary drastically.

In [2]:
np.random.seed(30)
import random as rn
rn.seed(30)
from keras import backend as K
import tensorflow as tf
tf.set_random_seed(30)
from keras.models import Sequential, Model
from keras.layers import Dense, GRU, Flatten, TimeDistributed, Flatten, BatchNormalization, Activation, Dropout
from keras.layers.convolutional import Conv2D, MaxPooling2D, AveragePooling2D
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau,EarlyStopping
from keras import optimizers
from keras.regularizers import l2
from keras.layers import LSTM, GRU, Bidirectional, SimpleRNN, RNN

Using TensorFlow backend.


In [3]:
#gpu_options = tf.GPUOptions(allow_growth=True)
#session = tf.InteractiveSession(config=tf.ConfigProto(gpu_options=gpu_options))

In this block, you read the folder names for training and validation. You also set the `batch_size` here. Note that you set the batch size in such a way that you are able to use the GPU in full capacity. You keep increasing the batch size until the machine throws an error.

In [3]:
train_doc = np.random.permutation(open('./Project_data/train.csv').readlines())
val_doc = np.random.permutation(open('./Project_data/val.csv').readlines())
batch_size = 20
print(train_doc.size)
print(val_doc.size)


663
100


# Generator
This is one of the most important part of the code. The overall structure of the generator has been given. In the generator, you are going to preprocess the images as you have images of 2 different dimensions as well as create a batch of video frames. You have to experiment with `img_idx`, `y`,`z` and normalization such that you get high accuracy.

In [4]:
import matplotlib.pyplot as plt

In [5]:
def transform_load_images(source_path,batch, folder_list, batch_size,validation):
    
    batch_data = np.zeros((batch_size,15,120,120,3))
    batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
    img_idx = [4,5,7,8,9,11,12,13,15,17,19,21,22,23,25]
    batch_data_warp = np.zeros((batch_size,15,120,120,3))
    batch_data_fliplr = np.zeros((batch_size,15,120,120,3))
    batch_labels_fliplr = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
    batch_labels_warp =  np.zeros((batch_size,5))
    dx, dy = np.random.randint(-1.7, 1.8, 2)
    M = np.float32([[1, 0, dx], [0, 1, dy]])
    t = np.random.permutation(folder_list)
    for folder in range(batch_size): # iterate over the batch_size
        imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
        for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
            
            image = cv2.imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item], cv2.IMREAD_COLOR)
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            #  print('inside 1 loop :img_idx')
            #shapes = set()
            if image.shape[0] != image.shape[1]:
                #shapes.add(image.shape)
                #print(shapes)
                #there are 2 different shapes(360,360) and (120,160)
                changed=image[0:120,20:140]
            else :    
                changed = cv2.resize(image, (120,120), interpolation = cv2.INTER_AREA)
                #changed = image[120:240,120:240]
            #and the conv3D will throw error if the inputs in a batch have different shapes
            #norm2_image = changed - np.percentile(changed,5)/ np.percentile(changed,95) - np.percentile(changed,5)
            norm2_image = changed - np.min(changed)/np.max(changed) - np.min(changed)
            
            #batch_data[folder,idx,:,:,0] = (changed[:,:,0])/255
            #batch_data[folder,idx,:,:,1] = (changed[:,:,1])/255
            #batch_data[folder,idx,:,:,2] = (changed[:,:,2])/255 
            
            batch_data[folder,idx] = norm2_image
            
            #Data with affine transformation
            batch_data_warp[folder,idx] = (cv2.warpAffine(changed, M, (changed.shape[0], changed.shape[1])))
            
            # Data with horizontal flip
            batch_data_fliplr[folder,idx]= np.fliplr(changed)
            
                    
        batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
        
        batch_labels_warp[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
        
        # Labeling data with horizobtal flip, right swipe becomes left swipe and viceversa
        if int(t[folder + (batch*batch_size)].strip().split(';')[2])==0:
                    batch_labels_fliplr[folder, 1] = 1
        elif int(t[folder + (batch*batch_size)].strip().split(';')[2])==1:
                    batch_labels_fliplr[folder, 0] = 1
                    
        else:
                    batch_labels_fliplr[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
                  
    
    batch_data_changed = np.append(batch_data, batch_data_warp, axis = 0)
    batch_data_changed = np.append(batch_data_changed, batch_data_fliplr, axis = 0)

    batch_labels_changed = np.append(batch_labels, batch_labels_warp, axis = 0) 
    batch_labels_changed = np.append(batch_labels_changed, batch_labels_fliplr, axis = 0)
    
    if validation:
        batch_data_changed=batch_data
        batch_labels_changed= batch_labels

    return  batch_data_changed, batch_labels_changed 

In [6]:
def generator(source_path, folder_list, batch_size,validation=False):
    print( 'Source path = ', source_path, '; batch size =', batch_size)
     #create a list of image numbers you want to use for a particular video
    while True:
        
               
        num_batches = ((len(folder_list) // batch_size) )
       # print(num_batches)
        cut_off = (num_batches * batch_size)
        #print(batch)
        for batch in range(num_batches): # we iterate over the number of batches
            
            yield transform_load_images(source_path,batch, folder_list, batch_size,validation) #you yield the batch_data and the batch_labels, remember what does yield do

        if (len(folder_list) != batch_size*num_batches):
            remaining_batch = (len(folder_list) - (num_batches*batch_size))
                     

            yield transform_load_images(source_path,batch, folder_list, remaining_batch,validation) #you yield the batch_data and the batch_labels, remember what does yield do    
        # write the code for the remaining data points which are left after full batches


Note here that a video is represented above in the generator as (number of images, height, width, number of channels). Take this into consideration while creating the model architecture.

In [7]:
curr_dt_time = datetime.datetime.now()
train_path = './Project_data/train'
val_path = './Project_data/val'
num_train_sequences = len(train_doc)
print('# training sequences =', num_train_sequences)
num_val_sequences = len(val_doc)
print('# validation sequences =', num_val_sequences)
num_epochs = 20 # choose the number of epochs
print ('# epochs =', num_epochs)

# training sequences = 663
# validation sequences = 100
# epochs = 20


## Model
Here you make the model using different functionalities that Keras provides. Remember to use `Conv3D` and `MaxPooling3D` and not `Conv2D` and `Maxpooling2D` for a 3D convolution model. You would want to use `TimeDistributed` while building a Conv2D + RNN model. Also remember that the last layer is the softmax. Design the network in such a way that the model is able to give good accuracy on the least number of parameters so that it can fit in the memory of the webcam.

Now that you have written the model, the next step is to `compile` the model. When you print the `summary` of the model, you'll see the total number of parameters you have to train.

Let us create the `train_generator` and the `val_generator` which will be used in `.fit_generator`.

In [8]:
train_generator = generator(train_path, train_doc, batch_size,False)
val_generator = generator(val_path, val_doc, batch_size,True)

In [9]:
model_name = 'model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.2,patience=5, verbose=1) 
earlystop = EarlyStopping( monitor="val_loss", min_delta=0,patience=10,verbose=1)
callbacks_list = [checkpoint, LR, earlystop]
# write the REducelronplateau code here
#callbacks_list = [checkpoint, LR]

The `steps_per_epoch` and `validation_steps` are used by `fit_generator` to decide the number of next() calls it need to make.

In [10]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

Let us now fit the model. This will start training the model and with the help of the checkpoints, you'll be able to save the model at the end of each epoch.

In [11]:
from keras.models import Sequential, Model
from keras.layers import Dense, GRU, Flatten, TimeDistributed, Flatten, BatchNormalization, Activation
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Conv3D, MaxPooling3D
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau,EarlyStopping
from keras.regularizers import l2
from keras import optimizers
from keras import regularizers

#write your model here

model=Sequential()



model.add(Conv3D(16, kernel_size=(3,3,3), input_shape = (15,120,120,3),padding='same'))
model.add(Activation('relu'))
model.add(BatchNormalization())


#model.add(Conv3D(16, kernel_size=(3,3,3),padding='same'))
#model.add(BatchNormalization())
#model.add(Activation('relu'))
model.add(MaxPooling3D(pool_size=(2,2,2)))
#model.add(Dropout(0.20))

model.add(Conv3D(32, kernel_size=(3,3,3),padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling3D(pool_size=(2,2,2)))
#model.add(Dropout(0.20))




model.add(Conv3D(64, kernel_size=(3,3,3),padding='same'))
model.add(Activation('relu'))
model.add(BatchNormalization())
#model.add(Dropout(0.20))
model.add(MaxPooling3D(pool_size=(2,2,2)))

#model.add(Dropout(0.20))
model.add(Conv3D(128, kernel_size=(3,3,3),padding='same'))
model.add(Activation('relu'))
model.add(BatchNormalization())
#model.add(Dropout(0.20))


#model.add(Dropout(0.20))
#model.add(MaxPooling3D(pool_size=(2,2,2)))



model.add(Flatten())
model.add(Dense(128,kernel_regularizer=l2(0.001)) )   
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(128,kernel_regularizer=l2(0.001)))    
model.add(Activation('relu'))
model.add(Dropout(0.25))
model.add(Dense(5))
model.add(Activation('softmax'))
#model.add(BatchNormalization(axis=1, momentum=0.99, epsilon=0.001, center=True, scale=True, beta_initializer='zeros', gamma_initializer='ones', moving_mean_initializer='zeros', moving_variance_initializer='ones', beta_regularizer=None, gamma_regularizer=None, beta_constraint=None, gamma_constraint=None))

#model.add(Dense(1, activation='relu', use_bias=True, kernel_initializer='glorot_normal', bias_initializer='zeros', kernel_regularizer=None, bias_regularizer=None, activity_regularizer=None, kernel_constraint=None, bias_constraint=None))

In [12]:
#optimiser = optimizers.Adam(lr=0.0003)
optimiser = optimizers.SGD(lr=0.0005, decay=1e-6, momentum=0.7, nesterov=True)
model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])

print (model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_1 (Conv3D)            (None, 15, 120, 120, 16)  1312      
_________________________________________________________________
activation_1 (Activation)    (None, 15, 120, 120, 16)  0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 15, 120, 120, 16)  64        
_________________________________________________________________
max_pooling3d_1 (MaxPooling3 (None, 7, 60, 60, 16)     0         
_________________________________________________________________
conv3d_2 (Conv3D)            (None, 7, 60, 60, 32)     13856     
_________________________________________________________________
batch_normalization_2 (Batch (None, 7, 60, 60, 32)     128       
_________________________________________________________________
activation_2 (Activation)    (None, 7, 60, 60, 32)     0         
__________

In [13]:
#resize not crop  #######don't run again stable model##############

## number of features((16,32,64,128))

model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  Source path =  ./Project_data/train ; batch size = 20
Epoch 1/20
./Project_data/val ; batch size = 20
34/34 [==============================] - 76s 2s/step - loss: 2.1425 - categorical_accuracy: 0.3247 - val_loss: 1.7431 - val_categorical_accuracy: 0.4400

Epoch 00001: saving model to model_init_2020-12-2218_35_53.288596/model-00001-2.12590-0.33283-1.74306-0.44000.h5
Epoch 2/20
34/34 [==============================] - 64s 2s/step - loss: 1.7887 - categorical_accuracy: 0.4104 - val_loss: 1.6102 - val_categorical_accuracy: 0.4200

Epoch 00002: saving model to model_init_2020-12-2218_35_53.288596/model-00002-1.78050-0.41227-1.61017-0.42000.h5
Epoch 3/20
34/34 [==============================] - 63s 2s/step - loss: 1.6651 - categorical_accuracy: 0.4663 - val_loss: 1.5936 - val_categorical_accuracy: 0.4000

Epoch 00003: saving model to model_init_2020-12-2218_35_53.288596/model-00003-1.65953-0.46958-1.59363-0.40000.h5
Epoch 4/20
34/34 [==============================] - 63s 2s/s

In [18]:
from keras.models import Sequential, Model
from keras.layers import Dense, GRU, Flatten, TimeDistributed, Flatten, BatchNormalization, Activation
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Conv3D, MaxPooling3D
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau,EarlyStopping
from keras.regularizers import l2
from keras import optimizers
from keras import regularizers

#write your model here

model=Sequential()



model.add(Conv3D(8, kernel_size=(3,3,3), input_shape = (15,120,120,3),padding='same'))
model.add(Activation('relu'))
model.add(BatchNormalization())


#model.add(Conv3D(16, kernel_size=(3,3,3),padding='same'))
#model.add(BatchNormalization())
#model.add(Activation('relu'))
model.add(MaxPooling3D(pool_size=(2,2,2)))
#model.add(Dropout(0.20))

model.add(Conv3D(16, kernel_size=(3,3,3),padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling3D(pool_size=(2,2,2)))
#model.add(Dropout(0.20))




model.add(Conv3D(32, kernel_size=(3,3,3),padding='same'))
model.add(Activation('relu'))
model.add(BatchNormalization())
#model.add(Dropout(0.20))
model.add(MaxPooling3D(pool_size=(2,2,2)))

#model.add(Dropout(0.20))
model.add(Conv3D(64, kernel_size=(3,3,3),padding='same'))
model.add(Activation('relu'))
model.add(BatchNormalization())
#model.add(Dropout(0.20))


#model.add(Dropout(0.20))
#model.add(MaxPooling3D(pool_size=(2,2,2)))



model.add(Flatten())
model.add(Dense(128,kernel_regularizer=l2(0.001)) )   
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(128,kernel_regularizer=l2(0.001)))    
model.add(Activation('relu'))
model.add(Dropout(0.25))
model.add(Dense(5))
model.add(Activation('softmax'))
#model.add(BatchNormalization(axis=1, momentum=0.99, epsilon=0.001, center=True, scale=True, beta_initializer='zeros', gamma_initializer='ones', moving_mean_initializer='zeros', moving_variance_initializer='ones', beta_regularizer=None, gamma_regularizer=None, beta_constraint=None, gamma_constraint=None))

#model.add(Dense(1, activation='relu', use_bias=True, kernel_initializer='glorot_normal', bias_initializer='zeros', kernel_regularizer=None, bias_regularizer=None, activity_regularizer=None, kernel_constraint=None, bias_constraint=None))

In [19]:
#optimiser = optimizers.Adam(lr=0.0003)
optimiser = optimizers.SGD(lr=0.0005, decay=1e-6, momentum=0.7, nesterov=True)
model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])

print (model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_9 (Conv3D)            (None, 15, 120, 120, 8)   656       
_________________________________________________________________
activation_15 (Activation)   (None, 15, 120, 120, 8)   0         
_________________________________________________________________
batch_normalization_9 (Batch (None, 15, 120, 120, 8)   32        
_________________________________________________________________
max_pooling3d_7 (MaxPooling3 (None, 7, 60, 60, 8)      0         
_________________________________________________________________
conv3d_10 (Conv3D)           (None, 7, 60, 60, 16)     3472      
_________________________________________________________________
batch_normalization_10 (Batc (None, 7, 60, 60, 16)     64        
_________________________________________________________________
activation_16 (Activation)   (None, 7, 60, 60, 16)     0         
__________

In [20]:
##################### regularizer #####################
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Epoch 1/20
34/34 [==============================] - 70s 2s/step - loss: 2.1997 - categorical_accuracy: 0.2669 - val_loss: 1.7458 - val_categorical_accuracy: 0.4100

Epoch 00001: saving model to model_init_2020-12-2218_35_53.288596/model-00001-2.21433-0.26244-1.74577-0.41000.h5
Epoch 2/20
34/34 [==============================] - 58s 2s/step - loss: 1.9086 - categorical_accuracy: 0.3358 - val_loss: 1.7444 - val_categorical_accuracy: 0.4800

Epoch 00002: saving model to model_init_2020-12-2218_35_53.288596/model-00002-1.91919-0.33032-1.74443-0.48000.h5
Epoch 3/20
34/34 [==============================] - 62s 2s/step - loss: 1.7900 - categorical_accuracy: 0.3873 - val_loss: 1.6714 - val_categorical_accuracy: 0.5500

Epoch 00003: saving model to model_init_2020-12-2218_35_53.288596/model-00003-1.78864-0.38864-1.67141-0.55000.h5
Epoch 4/20
34/34 [==============================] - 65s 2s/step - loss: 1.7408 - categorical_accuracy: 0.4059 - val_loss: 1.6543 - val_categorical_accuracy: 0.3800

E

In [22]:



model_LSMT = Sequential()

model_LSMT.add(TimeDistributed(Conv2D(16, (2, 2), padding='same'),
                 input_shape=(15,120,120,3)))
model_LSMT.add(Activation('relu'))
model_LSMT.add(BatchNormalization())
model_LSMT.add(TimeDistributed(MaxPooling2D(pool_size=(2, 2))))

model_LSMT.add(TimeDistributed(Conv2D(32, (2, 2))))
model_LSMT.add(Activation('relu'))
model_LSMT.add(BatchNormalization())
model_LSMT.add(TimeDistributed(MaxPooling2D(pool_size=(2, 2))))
#model_LSMT.add(Dropout(0.2))

model_LSMT.add(TimeDistributed(Conv2D(64, (2, 2), padding='same')))
model_LSMT.add(Activation('relu'))
model_LSMT.add(BatchNormalization())
model_LSMT.add(TimeDistributed(MaxPooling2D(pool_size=(2, 2))))

model_LSMT.add(TimeDistributed(Conv2D(128, (2, 2), padding='same')))
model_LSMT.add(Activation('relu'))
model_LSMT.add(BatchNormalization())
model_LSMT.add(TimeDistributed(MaxPooling2D(pool_size=(2, 2))))
#model_LSMT.add(Dropout(0.2))
model_LSMT.add(TimeDistributed(Conv2D(256, (2, 2), padding='same')))
model_LSMT.add(Activation('relu'))
model_LSMT.add(BatchNormalization())
model_LSMT.add(TimeDistributed(MaxPooling2D(pool_size=(2, 2))))

model_LSMT.add(TimeDistributed(Flatten()))
model_LSMT.add(LSTM(256, return_sequences=False, dropout=0.5))
model_LSMT.add(Dense(256))
model_LSMT.add(Activation('relu'))
model_LSMT.add(Dropout(0.25))

model_LSMT.add(Dense(256))
model_LSMT.add(Activation('relu'))
model_LSMT.add(Dropout(0.25))
model_LSMT.add(Dense(5))
model_LSMT.add(Activation('softmax'))

In [23]:
optimiser = optimizers.Adam(lr=0.0002)
#optimiser = optimizers.SGD(lr=0.001)
model_LSMT.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])

print (model_LSMT.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed_23 (TimeDis (None, 15, 120, 120, 16)  208       
_________________________________________________________________
activation_17 (Activation)   (None, 15, 120, 120, 16)  0         
_________________________________________________________________
batch_normalization_11 (Batc (None, 15, 120, 120, 16)  64        
_________________________________________________________________
time_distributed_24 (TimeDis (None, 15, 60, 60, 16)    0         
_________________________________________________________________
time_distributed_25 (TimeDis (None, 15, 59, 59, 32)    2080      
_________________________________________________________________
activation_18 (Activation)   (None, 15, 59, 59, 32)    0         
_________________________________________________________________
batch_normalization_12 (Batc (None, 15, 59, 59, 32)    128       
__________

In [24]:
model_LSMT.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Epoch 1/20
34/34 [==============================] - 73s 2s/step - loss: 1.5231 - categorical_accuracy: 0.3054 - val_loss: 1.2333 - val_categorical_accuracy: 0.5600

Epoch 00001: saving model to model_init_2020-12-2217_01_23.924378/model-00001-1.52359-0.30468-1.23334-0.56000.h5
Epoch 2/20
34/34 [==============================] - 59s 2s/step - loss: 1.3133 - categorical_accuracy: 0.4148 - val_loss: 1.1621 - val_categorical_accuracy: 0.6100

Epoch 00002: saving model to model_init_2020-12-2217_01_23.924378/model-00002-1.30385-0.42232-1.16213-0.61000.h5
Epoch 3/20
34/34 [==============================] - 63s 2s/step - loss: 1.1799 - categorical_accuracy: 0.4871 - val_loss: 1.1376 - val_categorical_accuracy: 0.5200

Epoch 00003: saving model to model_init_2020-12-2217_01_23.924378/model-00003-1.18447-0.48265-1.13761-0.52000.h5
Epoch 4/20
34/34 [==============================] - 63s 2s/step - loss: 1.0710 - categorical_accuracy: 0.5347 - val_loss: 1.0227 - val_categorical_accuracy: 0.5600

E

In [17]:
del model

In [21]:
del model_LSMT

In [29]:
model_GRU = Sequential()

model_GRU.add(TimeDistributed(Conv2D(32, (2, 2), padding='same'),
                 input_shape=(15,120,120,3)))
model_GRU.add(Activation('relu'))
model_GRU.add(BatchNormalization())
model_GRU.add(TimeDistributed(MaxPooling2D(pool_size=(2, 2))))

model_GRU.add(TimeDistributed(Conv2D(32, (2, 2))))
model_GRU.add(Activation('relu'))
model_GRU.add(BatchNormalization())
model_GRU.add(TimeDistributed(MaxPooling2D(pool_size=(2, 2))))
#model_LSMT.add(Dropout(0.2))

model_GRU.add(TimeDistributed(Conv2D(64, (2, 2), padding='same')))
model_GRU.add(Activation('relu'))
model_GRU.add(BatchNormalization())
model_GRU.add(TimeDistributed(MaxPooling2D(pool_size=(2, 2))))

model_GRU.add(TimeDistributed(Conv2D(128, (2, 2), padding='same')))
model_GRU.add(Activation('relu'))
model_GRU.add(BatchNormalization())
model_GRU.add(TimeDistributed(MaxPooling2D(pool_size=(2, 2))))
#model_LSMT.add(Dropout(0.2))
model_GRU.add(TimeDistributed(Conv2D(256, (2, 2), padding='same')))
model_GRU.add(Activation('relu'))
model_GRU.add(BatchNormalization())
model_GRU.add(TimeDistributed(MaxPooling2D(pool_size=(2, 2))))

model_GRU.add(TimeDistributed(Flatten()))
model_GRU.add(GRU(256, return_sequences=False, dropout=0.5))
model_GRU.add(Dense(256))
model_GRU.add(Activation('relu'))
model_GRU.add(Dropout(0.25))

model_GRU.add(Dense(256))
model_GRU.add(Activation('relu'))
model_GRU.add(Dropout(0.25))
model_GRU.add(Dense(5))
model_GRU.add(Activation('softmax'))

In [30]:
optimiser = optimizers.Adam(lr=0.0002)
#optimiser = optimizers.SGD(lr=0.001)
model_GRU.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])

print (model_GRU.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed_45 (TimeDis (None, 15, 120, 120, 16)  208       
_________________________________________________________________
activation_33 (Activation)   (None, 15, 120, 120, 16)  0         
_________________________________________________________________
batch_normalization_21 (Batc (None, 15, 120, 120, 16)  64        
_________________________________________________________________
time_distributed_46 (TimeDis (None, 15, 60, 60, 16)    0         
_________________________________________________________________
time_distributed_47 (TimeDis (None, 15, 59, 59, 32)    2080      
_________________________________________________________________
activation_34 (Activation)   (None, 15, 59, 59, 32)    0         
_________________________________________________________________
batch_normalization_22 (Batc (None, 15, 59, 59, 32)    128       
__________

In [31]:
model_GRU.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Epoch 1/20
34/34 [==============================] - 75s 2s/step - loss: 1.5610 - categorical_accuracy: 0.2872 - val_loss: 1.3259 - val_categorical_accuracy: 0.4400

Epoch 00001: saving model to model_init_2020-12-2217_01_23.924378/model-00001-1.56176-0.28607-1.32593-0.44000.h5
Epoch 2/20
34/34 [==============================] - 56s 2s/step - loss: 1.3674 - categorical_accuracy: 0.3817 - val_loss: 1.1655 - val_categorical_accuracy: 0.5000

Epoch 00002: saving model to model_init_2020-12-2217_01_23.924378/model-00002-1.34912-0.38562-1.16553-0.50000.h5
Epoch 3/20
34/34 [==============================] - 64s 2s/step - loss: 1.1672 - categorical_accuracy: 0.4945 - val_loss: 1.1927 - val_categorical_accuracy: 0.5300

Epoch 00003: saving model to model_init_2020-12-2217_01_23.924378/model-00003-1.16537-0.49573-1.19272-0.53000.h5
Epoch 4/20
34/34 [==============================] - 63s 2s/step - loss: 1.0399 - categorical_accuracy: 0.5619 - val_loss: 1.0246 - val_categorical_accuracy: 0.6300

E

In [ ]:
batch_data = np.zeros((20,30,120,3))
print(batch_data)

In [41]:
del  model_GRU

In [ ]:
# list all data in history
print(history.history.keys())
# summarize history for accuracy
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()